In [33]:
import pandas as pd
import numpy as np

In [34]:
Date='2023-03-03'
Line='3-10'
Model='LONGSP'

In [35]:
import pyodbc
import pandas as pd
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '192.168.101.219' 
database = 'DataforAnalysis' 
username = 'DATALYZER' 
password = 'NMB54321'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# select 26 rows from SQL table to insert in dataframe.
query = "SELECT [Projection1],[Datum_probe],[Max_force],[Set_Dim_A],[Set_Dim_B],[Set_Dim_C],[Pivot_Height]FROM [DataforAnalysis].[dbo].[DataML] WHERE [Model] = 'LONGSP' AND [Line] = '"+Line+"' AND [Date] BETWEEN '2022-11-01' and '"+Date+"'and ([Projection1] is not null and [Datum_probe] is not null and [Max_force] is not null and [Set_Dim_A] is not null and [Set_Dim_B] is not null and [Set_Dim_C] is not null and [Pivot_Height] is not null)and [Projection1] !='0' and [Max_force] >=100 and [Set_Dim_A] >=0.3775 and [Set_Dim_B] >=0.3775 and [Set_Dim_C] >=0.3775 and [Max_force] <= 3700.00 and [Set_Dim_A] <= 0.6537 and [Set_Dim_B] <= 0.6537 and  [Set_Dim_C] <= 0.6537;"
df = pd.read_sql(query, cnxn)
# print(df.head(100000))

C:\Users\IT\AppData\Local\Temp\ipykernel_15636\3403563190.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


In [36]:
datasets = df

In [37]:
max=(datasets['Projection1'])


In [38]:
datasets['Projection1']=pd.cut(datasets['Projection1'],bins=[-np.inf,0.4647,0.5664,np.inf],labels=['fail_low','pass','fail_high'])
datasets['Projection1'].replace('fail_low','fail',inplace=True)
datasets['Projection1'].replace('fail_high','fail',inplace=True)

In [39]:
datasets

,Projection1,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height
0,fail,358.459991,1180,0.5686,0.5209,0.5568,-9.416
1,fail,358.459991,1180,0.5692,0.5219,0.5555,-9.417
2,fail,358.459991,1180,0.5582,0.5395,0.5529,-9.417
3,fail,358.459991,1180,0.5608,0.5354,0.5533,-9.416
4,fail,358.459991,1180,0.5683,0.5264,0.5502,-9.416
...,...,...,...,...,...,...,...
645501,pass,485.049988,918,0.4996,0.4976,0.5133,-9.383
645502,pass,465.519989,1168,0.5041,0.5077,0.5251,-9.382
645503,pass,483.089996,1248,0.5073,0.5004,0.5091,-9.381
645504,pass,464.600006,1314,0.5056,0.4967,0.5137,-9.381


In [40]:
from collections import Counter
print(Counter(datasets['Projection1']))

Counter({'pass': 641408, 'fail': 4098})


In [41]:
from imblearn.over_sampling import SMOTE
# over_sampling คือการทำค่า fail 1284 ให้เท่ากับค่า pass 271k เพื่อให้ data ในการ train เพิ่มมากขึ้น
from collections import Counter

k = 3
X = datasets.loc[:, datasets.columns != 'Projection1']
y = datasets.Projection1

# sampling_strategy=0.1 oversampling 10%
sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
X_res, y_res = sm.fit_resample(X, y)

datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

In [42]:
from collections import Counter
print(Counter(datasets['Projection1']))

Counter({'fail': 641408, 'pass': 641408})


In [43]:
datasets

,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height,Projection1
0,358.459991,1180.0,0.568600,0.520900,0.556800,-9.416000,fail
1,358.459991,1180.0,0.569200,0.521900,0.555500,-9.417000,fail
2,358.459991,1180.0,0.558200,0.539500,0.552900,-9.417000,fail
3,358.459991,1180.0,0.560800,0.535400,0.553300,-9.416000,fail
4,358.459991,1180.0,0.568300,0.526400,0.550200,-9.416000,fail
...,...,...,...,...,...,...,...
1282811,464.970001,1009.0,0.512888,0.502293,0.533095,-9.383000,fail
1282812,467.428445,1326.550072,0.512335,0.500430,0.534725,-9.387550,fail
1282813,468.165326,1450.047515,0.517581,0.497805,0.526821,-9.391691,fail
1282814,463.501623,554.296516,0.507492,0.497728,0.490635,-9.379259,fail


In [44]:
Datum_3LCL = 124.99
Datum_2LCL = 249.99
Datum_LCL = 374.99
Datum_CL = 499.99
Datum_UCL = 624.99
Datum_2UCL = 749.99
Datum_3UCL = 875

Max_3LCL = 299.99
Max_2LCL = 833.32
Max_LCL = 1366.66
Max_CL = 1899.99
Max_UCL = 2433.32
Max_2UCL = 2966.66
Max_3UCL = 3500

Set_Dim_A_3LCL = 0.4774
Set_Dim_A_2LCL = 0.4901
Set_Dim_A_LCL = 0.5028
Set_Dim_A_CL = 0.5155
Set_Dim_A_UCL = 0.5282
Set_Dim_A_2UCL = 0.5409
Set_Dim_A_3UCL = 0.5537


Set_Dim_B_3LCL = 0.4774
Set_Dim_B_2LCL = 0.4901
Set_Dim_B_LCL = 0.5058
Set_Dim_B_CL = 0.5155
Set_Dim_B_UCL = 0.5282
Set_Dim_B_2UCL =0.5409
Set_Dim_B_3UCL = 0.5537

Set_Dim_C_3LCL = 0.4774
Set_Dim_C_2LCL = 0.4901
Set_Dim_C_LCL = 0.5028
Set_Dim_C_CL = 0.5155
Set_Dim_C_UCL = 0.5282
Set_Dim_C_2UCL =0.5409
Set_Dim_C_3UCL = 0.5537

Pivot_Height_3LCL = -9.4030
Pivot_Height_2LCL = -9.3945
Pivot_Height_LCL = -9.3860
Pivot_Height_CL = -9.3776
Pivot_Height_UCL = -9.3691
Pivot_Height_2UCL = -9.3606
Pivot_Height_3UCL = -9.3523


datasets['Datum_probe']=pd.cut(datasets['Datum_probe'],bins=[-np.inf, Datum_3LCL, Datum_2LCL, Datum_LCL, Datum_CL, Datum_UCL, Datum_2UCL, Datum_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Max_force']=pd.cut(datasets['Max_force'],bins=[-np.inf, Max_3LCL, Max_2LCL, Max_LCL, Max_CL, Max_UCL, Max_2UCL, Max_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_A']=pd.cut(datasets['Set_Dim_A'],bins=[-np.inf, Set_Dim_A_3LCL, Set_Dim_A_2LCL, Set_Dim_A_LCL, Set_Dim_A_CL, Set_Dim_A_UCL, Set_Dim_A_2UCL, Set_Dim_A_3UCL,np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_B']=pd.cut(datasets['Set_Dim_B'],bins=[-np.inf, Set_Dim_B_3LCL, Set_Dim_B_2LCL, Set_Dim_B_LCL, Set_Dim_B_CL, Set_Dim_B_UCL, Set_Dim_B_2UCL, Set_Dim_B_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Set_Dim_C']=pd.cut(datasets['Set_Dim_C'],bins=[-np.inf, Set_Dim_C_3LCL, Set_Dim_C_2LCL, Set_Dim_C_LCL, Set_Dim_C_CL, Set_Dim_C_UCL, Set_Dim_C_2UCL, Set_Dim_C_3UCL, np.inf],labels=['A','B','C','D','E','F','G','H'])

datasets['Pivot_Height']=pd.cut(datasets['Pivot_Height'],bins=[-np.inf, Pivot_Height_3LCL, Pivot_Height_2LCL, Pivot_Height_LCL, Pivot_Height_CL, Pivot_Height_UCL, Pivot_Height_2UCL, Pivot_Height_3UCL,np.inf],labels=['A','B','C','D','E','F','G','H'])


In [45]:
datasets

,Datum_probe,Max_force,Set_Dim_A,Set_Dim_B,Set_Dim_C,Pivot_Height,Projection1
0,C,C,H,E,H,A,fail
1,C,C,H,E,H,A,fail
2,C,C,H,F,G,A,fail
3,C,C,H,F,G,A,fail
4,C,C,H,E,G,A,fail
...,...,...,...,...,...,...,...
1282811,D,C,D,C,F,D,fail
1282812,D,C,D,C,F,C,fail
1282813,D,D,E,C,E,C,fail
1282814,D,B,D,C,C,D,fail


In [46]:
from pyarc import CBA, TransactionDB
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report

train, test = train_test_split(datasets, test_size=0.1)

txns_train = TransactionDB.from_DataFrame(train)
txns_test = TransactionDB.from_DataFrame(test)

cba = CBA (support = 0.09, confidence = 0.8, algorithm ='m1')
cba.fit(txns_train)

y_pred = cba.predict(txns_test)
y_test = test['Projection1']
print ((classification_report(y_test,y_pred)))


              precision    recall  f1-score   support

        fail       0.92      0.30      0.46     64152
        pass       0.58      0.97      0.73     64130

    accuracy                           0.64    128282
   macro avg       0.75      0.64      0.59    128282
weighted avg       0.75      0.64      0.59    128282



In [47]:
datasp =((classification_report(y_test,y_pred)))

In [48]:
datasp.split("\n")

['              precision    recall  f1-score   support',
 '',
 '        fail       0.92      0.30      0.46     64152',
 '        pass       0.58      0.97      0.73     64130',
 '',
 '    accuracy                           0.64    128282',
 '   macro avg       0.75      0.64      0.59    128282',
 'weighted avg       0.75      0.64      0.59    128282',
 '']

In [49]:
line=['3-10'       ,datasp[14:23],datasp[27:33],datasp[35:43],datasp[46:53]]
print(line)
fail=[datasp[63:67] ,datasp[74:78],datasp[84:88],datasp[94:98],datasp[103:108]]
print(fail)
pass1=[datasp[117:121],datasp[128:132],datasp[138:142],datasp[148:152],datasp[157:162]]
print(pass1)
accuracy=[datasp[168:176],[]             ,[]             ,datasp[203:207],datasp[211:217]]
print(accuracy)
macro_avg=[datasp[221:230],datasp[237:241],datasp[247:251],datasp[257:261],datasp[265:271]]
print(macro_avg)
weighted_avg=[datasp[272:284],datasp[291:295],datasp[301:305],datasp[311:315],datasp[319:325]]
print(weighted_avg)

['3-10', 'precision', 'recall', 'f1-score', 'support']
['fail', '0.92', '0.30', '0.46', '64152']
['pass', '0.58', '0.97', '0.73', '64130']
['accuracy', [], [], '0.64', '128282']
['macro avg', '0.75', '0.64', '0.59', '128282']
['weighted avg', '0.75', '0.64', '0.59', '128282']


In [50]:
from pandas import DataFrame

data ={
     'Date':Date,
    'Line':Line,
    'Model':Model,
'Details':[(datasp[63:67]),(datasp[117:121]),(datasp[168:176]),(datasp[221:230]),(datasp[272:284])
],
'precision':[datasp[74:78],datasp[128:132],(''),datasp[128:132],datasp[237:241]
],
'recall':[datasp[84:88],datasp[138:142],(''),datasp[247:251],datasp[301:305]
],
'f1-score':[datasp[94:98],datasp[148:152],(datasp[203:207]),datasp[257:261],datasp[311:315]
],

'support':[datasp[103:108],datasp[157:162],datasp[211:217],datasp[265:271],datasp[319:325]
],
}
df = pd.DataFrame(data,columns=['Date','Line','Model','Details','precision','recall','f1-score','support'])
print("displaying the dataframe with data as record")
print(df)
print("-------------------------------Converting into csv file with a new a new file name ------------------------------------")
df.to_csv("C:\\Users\\IT\\Desktop\\Date ML\\Accuracy ML\\LSP3-10 Accuracy.csv")
df=pd.read_csv("C:\\Users\\IT\\Desktop\\Date ML\\Accuracy ML\\LSP3-10 Accuracy.csv")
print(df)

displaying the dataframe with data as record
         Date  Line   Model       Details precision recall f1-score support
0  2023-03-03  3-10  LONGSP          fail      0.92   0.30     0.46   64152
1  2023-03-03  3-10  LONGSP          pass      0.58   0.97     0.73   64130
2  2023-03-03  3-10  LONGSP      accuracy                      0.64  128282
3  2023-03-03  3-10  LONGSP     macro avg      0.58   0.64     0.59  128282
4  2023-03-03  3-10  LONGSP  weighted avg      0.75   0.64     0.59  128282
-------------------------------Converting into csv file with a new a new file name ------------------------------------
   Unnamed: 0        Date  Line   Model       Details  precision  recall  \
0           0  2023-03-03  3-10  LONGSP          fail       0.92    0.30   
1           1  2023-03-03  3-10  LONGSP          pass       0.58    0.97   
2           2  2023-03-03  3-10  LONGSP      accuracy        NaN     NaN   
3           3  2023-03-03  3-10  LONGSP     macro avg       0.58    0.64   

In [51]:
cba1 =cba.clf.rules

In [52]:
type(cba1)

list

In [53]:
data =''
aa =''
dd=[]
for  x in range(len(cba1)):
    aa += str (cba1[x]).replace("{",'').replace("}",'').replace("Projection1=",'')
    dd=aa.split("CAR")
print(dd)

['', ' Set_Dim_B=C,Set_Dim_C=F => fail sup: 0.10 conf: 0.94 len: 3, id: 26', ' Max_force=C,Set_Dim_C=F => fail sup: 0.10 conf: 0.93 len: 3, id: 24', ' Set_Dim_C=F,Datum_probe=D,Pivot_Height=D => fail sup: 0.11 conf: 0.91 len: 4, id: 27']


In [54]:
fid=[]
fid01=[]
Sup=[]
Conf=[]
for op in range(len(dd)):
    dat=(dd[op].split())
    fid+=dat[0:1]
    fid01+=dat[2:3]
    Sup+=dat[4:5]
    Conf+=dat[6:7]

In [55]:
my_array=[]
ok =[]
i=0
while i < len(fid):
    my_array+= fid[i].split()
    ok +=[my_array[i]+','+'Null=Null'+','+'Null=Null'+','+'Null=Null'+','+'Null=Null']
    i=i+1

In [56]:
Sp=[]
KPIV1=[]
KPIV2=[]
KPIV3=[]
KPIV4=[]
for i in ok:
    Sp1=i.split(',')
    KPIV1+=[Sp1[0]]
    KPIV2+=[Sp1[1]]
    KPIV3+=[Sp1[2]]
    KPIV4+=[Sp1[3]]

KPIVOK=[KPIV1],[KPIV2],[KPIV3],[KPIV4]

In [57]:
dataok=[]
for i in range(len(KPIVOK)):
    dataok+=(KPIVOK[i])

In [58]:
sp1=[]
for i in (dataok[0]):
    sp1+=[(i.split('='))]

sp2=[]
for i in (dataok[1]):
    sp2+=[(i.split('='))]

sp3=[]
for i in (dataok[2]):
    sp3+=[(i.split('='))]

sp4=[]
for i in (dataok[3]):
    sp4+=[(i.split('='))]

In [59]:
Cl1=[]
for i in sp1:
    Cl1+=([i[0]])

Cl2=[]
for i in sp1:
    Cl2+=([i[1]])

Cl3=[]
for i in sp2:
    Cl3+=([i[0]])

Cl4=[]
for i in sp2:
    Cl4+=([i[1]])

Cl5=[]
for i in sp3:
    Cl5+=([i[0]])

Cl6=[]
for i in sp3:
    Cl6+=([i[1]])

In [60]:
import pandas as pd

data ={
'Date':Date,
'betweenDate':'-',
'Model':Model,
'Line':Line,
'Cl1':Cl1,
'Cl2':Cl2,
'Cl3':Cl3,
'Cl4':Cl4,
'Cl5':Cl5,
'Cl6':Cl6,
'Projection1':fid01,
'Support':Sup,
'Confidence':Conf,}
df = pd.DataFrame(data)
df.set_index('Date')

df.loc[df['Cl1'] == 'Datum_probe', 'Datum_probe'] = df['Cl2']
df.loc[df['Cl3'] == 'Datum_probe', 'Datum_probe'] = df['Cl4']
df.loc[df['Cl5'] == 'Datum_probe', 'Datum_probe'] = df['Cl6']

df.loc[df['Cl1'] == 'Max_force', 'Max_force'] = df['Cl2']
df.loc[df['Cl3'] == 'Max_force', 'Max_force'] = df['Cl4']
df.loc[df['Cl5'] == 'Max_force', 'Max_force'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_A', 'Set_Dim_A'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_B', 'Set_Dim_B'] = df['Cl6']

df.loc[df['Cl1'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl2']
df.loc[df['Cl3'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl4']
df.loc[df['Cl5'] == 'Set_Dim_C', 'Set_Dim_C'] = df['Cl6']

df.loc[df['Cl1'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl2']
df.loc[df['Cl3'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl4']
df.loc[df['Cl5'] == 'Pivot_Height', 'Pivot_Height'] = df['Cl6']


df = pd.DataFrame(df,columns=['Date','betweenDate','Model','Line','Datum_probe','Max_force','Set_Dim_A','Set_Dim_B','Set_Dim_C','Pivot_Height','Projection1','Support','Confidence'])
print("displaying the dataframe with data as record")
print(df)
print("-------------------------------Converting into csv file with a new a new file name ------------------------------------")
df.to_csv("C:\\Users\\IT\\Desktop\\Date ML\\3-10 KPIV.csv")
df=pd.read_csv("C:\\Users\\IT\\Desktop\\Date ML\\3-10 KPIV.csv")
print(df)



displaying the dataframe with data as record
         Date betweenDate   Model  Line Datum_probe Max_force Set_Dim_A  \
0  2023-03-03           -  LONGSP  3-10         NaN       NaN       NaN   
1  2023-03-03           -  LONGSP  3-10         NaN         C       NaN   
2  2023-03-03           -  LONGSP  3-10           D       NaN       NaN   

  Set_Dim_B Set_Dim_C Pivot_Height Projection1 Support Confidence  
0         C         F          NaN        fail    0.10       0.94  
1       NaN         F          NaN        fail    0.10       0.93  
2       NaN         F            D        fail    0.11       0.91  
-------------------------------Converting into csv file with a new a new file name ------------------------------------
   Unnamed: 0        Date betweenDate   Model  Line Datum_probe Max_force  \
0           0  2023-03-03           -  LONGSP  3-10         NaN       NaN   
1           1  2023-03-03           -  LONGSP  3-10         NaN         C   
2           2  2023-03-03      